In [50]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import torch
from transformers import GPTNeoForSequenceClassification, GPTNeoForCausalLM, GPT2Tokenizer
from model import GPTNeoForSequenceClassificationBinary
import lightning.pytorch as pl
from model import GPTSequenceClassifiationModule
model = GPTSequenceClassifiationModule.load_from_checkpoint("/root/autodl-tmp/runs/model.ckpt")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['score.weight', 'transformer.h.23.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'transformer.h.1.attn.attention.bias', 'transformer.h.13.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'transformer.h.17.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.21.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.19.attn.attention.bias', 'transformer.h.3.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Changing layer  23
Changing layer  22
Changing layer  21
Changing layer  20


In [51]:
model=model.eval()
model=model.cuda()

In [52]:
from dataset import get_taxonomy_dataset, get_taxonomy_dataset_binary
dataset = get_taxonomy_dataset_binary('/root/Taxonomy-GPT/data/test_jun_21.csv', entire_dataset=True,remove_columns=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ddf3587681cde657/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


Map:   0%|          | 0/4438 [00:00<?, ? examples/s]

In [53]:
dataset

{'test': Dataset({
     features: ['flag', 'input_ids', 'labels'],
     num_rows: 4438
 })}

In [54]:
def format_dataset(example,tokenizer):
    prompt_template = "I am doing the taxonomy research. I think {child} is a subtopic of {parent}"
    prompt = prompt_template.format(child=example['child'], parent=example['parent'])
    example['input_ids'] = tokenizer.encode(prompt)
    return example
def test_dataloader(str_path,batch_size):
    dataset= get_taxonomy_dataset_binary(str_path,entire_dataset=True)
    dataset = dataset['test']
    dataset = dataset.map(lambda example: format_dataset(example,tokenizer))
    dataset = dataset.remove_columns(['child', 'parent', 'group'])

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=DataCollatorWithPadding(tokenizer),
        num_workers=4,
    )

In [55]:
from torch.utils.data import DataLoader
from transformers.data.data_collator import DataCollatorWithPadding
from tqdm.notebook import tqdm
import torch 

results = []
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer)

testdataloader = test_dataloader('/root/Taxonomy-GPT/data/test_jun_21.csv', 64)
with torch.no_grad():
    for step, batch in enumerate(tqdm(testdataloader)):
        batch = batch.to('cuda')
        output = model(batch['input_ids'], batch['attention_mask'])
        predicted_labels = torch.sigmoid(output)
        results.extend(predicted_labels.cpu().detach().numpy().tolist())

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-ddf3587681cde657/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-ddf3587681cde657/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-417e06c88f2707ad.arrow


Map:   0%|          | 0/4438 [00:00<?, ? examples/s]

  0%|          | 0/70 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [56]:

import pandas as pd
df = pd.read_csv('/root/Taxonomy-GPT/data/test_jun_21.csv')
df['predict'] = results
df['pred_flag']=[True if i>0.5 else False for i in df['predict']]

In [57]:
df.to_csv('/notebooks/test_all_res_jun_21.csv', index=False)

In [58]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_test=df['flag']
y_pred=df['pred_flag']
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))
print(accuracy_score(y_test, y_pred))

0.7982174594395421
0.8018074592716673
0.7986241427547852
0.798783235691753


maximum spanning tree

In [59]:
import networkx as nx
df_t=df
forest=[]
for g in set(list(df_t.group)):
    df_tree=df_t[df_t.group==g]
    graph = nx.DiGraph()
    parents=df_tree['parent'].tolist()
    children=df_tree['child'].tolist()
    probabilities=df_tree['predict'].tolist()
    nodes = set(parents + children)
    for node in nodes:
        graph.add_node(node)
    for i in range(len(parents)):
        graph.add_edge(parents[i], children[i], weight=probabilities[i])
    T=nx.maximum_spanning_arborescence(graph)
    df=nx.to_pandas_edgelist(T)
    df['group']=g
    forest.append(df)

In [60]:
res_v2=pd.concat(forest, ignore_index=True)

In [61]:
res_v2

,source,target,weight,group
0,explosion,airburst,0.963787,647
1,explosion,blowback,0.692026,647
2,explosion,big_bang,0.915881,647
3,explosion,inflation,0.717169,647
4,explosion,blast,0.863965,647
...,...,...,...,...
2221,polyunsaturated_fatty_acid,linoleic_acid,0.837743,760
2222,polyunsaturated_fatty_acid,omega-3_fatty_acid,0.384040,760
2223,polyunsaturated_fatty_acid,omega-6_fatty_acid,0.386586,760
2224,omega-3_fatty_acid,eicosapentaenoic_acid,0.584579,760


In [62]:
res_v2.columns=['parent','child','weight','group']
res_v2['compare']=res_v2['parent']+res_v2['child']+res_v2['group'].astype(str)
res_v2['group'].value_counts()

group
677    45
666    39
702    39
697    38
712    37
       ..
744    10
648    10
738    10
737    10
647    10
Name: count, Length: 114, dtype: int64

In [63]:
df_actual=pd.read_csv('/root/Taxonomy-GPT/data/bansal_wordnet_true_pairs.csv')
df_actual=df_actual[df_actual['group']>= 647]
df_actual['compare']=df_actual['parent']+df_actual['child']+df_actual['group'].astype(str)

In [64]:
df_actual

,parent,child,group,type,compare
12646,explosion,airburst,647,test,explosionairburst647
12647,explosion,blowback,647,test,explosionblowback647
12648,explosion,big_bang,647,test,explosionbig_bang647
12649,explosion,inflation,647,test,explosioninflation647
12650,explosion,blast,647,test,explosionblast647
...,...,...,...,...,...
14867,polyunsaturated_fatty_acid,omega-6_fatty_acid,760,test,polyunsaturated_fatty_acidomega-6_fatty_acid760
14868,omega-3_fatty_acid,docosahexaenoic_acid,760,test,omega-3_fatty_aciddocosahexaenoic_acid760
14869,omega-3_fatty_acid,eicosapentaenoic_acid,760,test,omega-3_fatty_acideicosapentaenoic_acid760
14870,omega-3_fatty_acid,alpha-linolenic_acid,760,test,omega-3_fatty_acidalpha-linolenic_acid760


In [65]:
res_v2

,parent,child,weight,group,compare
0,explosion,airburst,0.963787,647,explosionairburst647
1,explosion,blowback,0.692026,647,explosionblowback647
2,explosion,big_bang,0.915881,647,explosionbig_bang647
3,explosion,inflation,0.717169,647,explosioninflation647
4,explosion,blast,0.863965,647,explosionblast647
...,...,...,...,...,...
2221,polyunsaturated_fatty_acid,linoleic_acid,0.837743,760,polyunsaturated_fatty_acidlinoleic_acid760
2222,polyunsaturated_fatty_acid,omega-3_fatty_acid,0.384040,760,polyunsaturated_fatty_acidomega-3_fatty_acid760
2223,polyunsaturated_fatty_acid,omega-6_fatty_acid,0.386586,760,polyunsaturated_fatty_acidomega-6_fatty_acid760
2224,omega-3_fatty_acid,eicosapentaenoic_acid,0.584579,760,omega-3_fatty_acideicosapentaenoic_acid760


In [66]:
df_actual.merge(res_v2, on='compare',how='left').to_csv('/root/Taxonomy-GPT/data/group_wise_jun_21.csv')

In [67]:
len(df_actual.merge(res_v2, on='compare'))/len(df_actual)

0.9227313566936208